Trying to get work the heatmap in an example script


# Import Required Libraries
Import the necessary libraries, including TensorFlow, Keras, Matplotlib, and NumPy.

In [ ]:
import keras
from keras import layers, models, utils, datasets

import matplotlib.pyplot as plt
import numpy as np

keras.version()

from keras import backend
print("Keras is using the backend:", backend.backend())

# Load and Preprocess Data
Load the dataset and preprocess it for training the CNN model.

In [ ]:
# Load the dataset (using CIFAR-10 dataset as an example)
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# Normalize the images to a range of 0 to 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

# Print the shape of the data to verify
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Build and Train the CNN Model
Define the CNN model architecture, compile it, and train it using the training data.

In [ ]:


height, width, channels = 32, 32, 3

inputs = keras.Input(shape=(height, width, channels))

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_1'),
    layers.MaxPooling2D((2, 2), name='maxpool2d_1'),
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'),
    layers.MaxPooling2D((2, 2), name='maxpool2d_2'),
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_3'),
    layers.Flatten(name='flatten'),
    layers.Dense(64, activation='relu', name='dense_1'),
    layers.Dense(10, activation='softmax', name='dense_2')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=32,
    validation_data=(X_test, y_test)
)

In [ ]:
model.summary()

In [ ]:
# Call the model symbolically to define its graph
outputs = model(inputs)

# Now you can access model.output
print("Model Output:", model.output)

# Generate Heatmap
Use the Grad-CAM technique to generate a heatmap for a specific input image to visualize the regions of the image that the CNN model focuses on.

In [ ]:
# Generate Heatmap
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Select an image from the test set
img = X_test[0]
img = np.expand_dims(img, axis=0)

# Get the model's predictions
preds = model.predict(img)
pred_class = np.argmax(preds[0])

# Get the output of the last convolutional layer
last_conv_layer = model.get_layer('conv2d_3')
print("Model Output:", model.output)
heatmap_model = keras.Model([model.inputs], [last_conv_layer.output, model.output])

# Compute the gradient of the top predicted class with respect to the output feature map of the last conv layer
with tf.GradientTape() as tape:
    conv_outputs, predictions = heatmap_model(img)
    loss = predictions[:, pred_class]

# Extract the gradients of the top predicted class with respect to the output feature map of the last conv layer
grads = tape.gradient(loss, conv_outputs)

# Compute the guided gradients
guided_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# Compute the heatmap
conv_outputs = conv_outputs[0]
heatmap = keras.reduce_mean(keras.multiply(guided_grads, conv_outputs), axis=-1)

# Apply ReLU to the heatmap
heatmap = np.maximum(heatmap, 0)

# Normalize the heatmap
heatmap /= np.max(heatmap)

# Resize the heatmap to match the size of the original image
heatmap = keras.image.resize(heatmap[..., np.newaxis], (32, 32)).numpy()

# Plot the heatmap
plt.matshow(heatmap.squeeze())
plt.show()

# Superimpose the heatmap on the original image
img = X_test[0]
heatmap = np.uint8(255 * heatmap.squeeze())
heatmap = np.expand_dims(heatmap, axis=-1)
superimposed_img = heatmap * 0.4 + img

# Plot the superimposed image
plt.imshow(superimposed_img / 255.0)
plt.show()